***Installing dependencies and Importing Libraries***

In [1]:
pip install selenium

     ---------------------------------------- 6.9/6.9 MB 7.9 MB/s eta 0:00:00
     ------------------------------------- 384.9/384.9 kB 12.1 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [268]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import sqlite3

In [271]:
from_city = input("Flying from: ")
to_city = input("Flying to: ")
day = input("What date (dd/mm/yyyy): ")

Flying from: mco
Flying to: hyd
What date (dd/mm/yyyy): 09/05/2023


In [308]:
url = f"https://www.expedia.ie/Flights-Search?trip=oneway&leg1=from:{from_city},to:{to_city},departure:{day}TANYT&passengers=adults:1,children:0,seniors:0,infantinlap:Y&options=cabinclass:economy&mode=search&origref=www.expedia.ie"

print(f"URL: {url}")
  

URL: https://www.expedia.ie/Flights-Search?trip=oneway&leg1=from:mco,to:hyd,departure:09/05/2023TANYT&passengers=adults:1,children:0,seniors:0,infantinlap:Y&options=cabinclass:economy&mode=search&origref=www.expedia.ie


In [309]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url)
time.sleep(10)
soup = BeautifulSoup(driver.page_source,  "lxml")
driver.quit()

In [310]:
results = soup.find_all('div',{"class":"uitk-card uitk-card-roundcorner-all uitk-spacing uitk-spacing-margin-blockend-three uitk-card-has-primary-theme"})
#print(results)

In [311]:
import pandas as pd

arrival_=[]
depature=[]
duration_=[]
layovers_=[]
airlines_=[]
price_=[]
for result in results:
    arrival = result.find('div',{"data-test-id":"arrival-time"})
    arrival_text = arrival.text.strip()
    a = arrival_text.split('-')
    arrival_.append(a[0])
    a1=a[1].split('+')
    depature.append(a1[0])
    duration = result.find('div',{"data-test-id":"journey-duration"})
    duration_text = duration.text.strip()
    d= duration_text.split('(')
    duration_.append(d[0])
    layovers = result.find('div',{'data-test-id':"layovers"})
    layovers_text=layovers.text.strip()
    layovers_.append(layovers_text)
    airlines=result.find('div',{'data-test-id':'flight-operated'})
    airlines_text=airlines.text.strip()
    al=airlines_text.split("•")
    airlines_.append(al[0])
    price = result.find('div',{'data-test-id':'price-column'})
    price_text=price.text.strip()
    p=price_text.split("€")
    text=p[1].replace(",","")
    price_.append(float(text)*1.20)

In [312]:
data ={"arrival": arrival_,"depature" : depature, "duration" : duration_ , "layovers" : layovers_ , "airlines": airlines_ , "price" :price_ }

In [313]:
data = pd.DataFrame(data)

In [314]:
data

,arrival,depature,duration,layovers,airlines,price
0,15:38,03:15,26h 7m,4h 11m in New York (JFK) • 2h 25m in Abu Dhabi...,Multiple airlines,787.2
1,12:02,03:15,29h 43m,7h 50m in New York (JFK) • 2h 25m in Abu Dhabi...,Multiple airlines,787.2
2,12:02,03:15,29h 43m,7h 50m in New York (JFK) • 2h 25m in Abu Dhabi...,Etihad Airways,787.2
3,11:00,03:15,30h 45m,8h 53m in New York (JFK) • 2h 25m in Abu Dhabi...,Multiple airlines,787.2
4,08:40,03:15,33h 5m,11h 13m in New York (JFK) • 2h 25m in Abu Dhab...,Etihad Airways,787.2


In [315]:
url1= f"https://www.studentuniverse.com/flights/1/{from_city}/{to_city}/2023-05-09?flexible=false&premiumCabins=false&source=cmsPage"

In [316]:
print(f"URL: {url1}")

URL: https://www.studentuniverse.com/flights/1/mco/hyd/2023-05-09?flexible=false&premiumCabins=false&source=cmsPage


In [317]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url1)
time.sleep(10)
soup = BeautifulSoup(driver.page_source,  "lxml")
driver.quit()

In [318]:
results_su= soup.find_all('div',{"class":"itinerary__container"})
#print(results)

In [319]:
arrival_=[]
depature=[]
duration_=[]
layovers_=[]
airlines_=[]
price_=[]

for result in results_su:
    #print(result)
    airlines=result.find('p',{"class":"mb-0 ng-binding ng-scope"})
    if(airlines == None):
        continue
    airlines_text=airlines.text.strip()
    airlines_.append(airlines_text)
    duration=result.find('span',{"id":"duration0"})
    duration_text=duration.text.strip()
    duration_.append(duration_text)
    departure = result.find('span',{"id":"departureTime0"})
    departure_text = departure.text.strip()
    depature.append(departure_text)
    arrival = result.find('span',{"id":"arrivalTime0"})
    arrival_text = arrival.text.strip()
    arrival_.append(arrival_text)
    layovers = result.find('div',{"class":"leg-summary__connections d-none d-sm-block ng-scope"})
    layovers_text = layovers.text.strip()
    layovers_.append(layovers_text)
    price = result.find('span',{"context":"priceBreakdown"})
    price_text = price.text.strip()
    price_text=price_text.split('$')
    text=price_text[1].replace(",","")
    price_.append(float(text))

In [320]:
data_su ={"arrival": arrival_,"depature" : depature, "duration" : duration_ , "layovers" : layovers_ , "airlines": airlines_ , "price" :price_ }

In [321]:
data_su = pd.DataFrame(data_su)

In [322]:
data_su

,arrival,depature,duration,layovers,airlines,price
0,6:55a,12:45p,32h 40m,"(BOS, AMS, BOM)",KLM,789.0
1,6:55a,11:20a,34h 5m,"(JFK, CDG, BOM)",Air France,792.0
2,8:20a,11:20a,35h 30m,"(JFK, CDG, BOM)",Air France,792.0
3,3:05a,5:57p,23h 38m,"(EWR, DXB)",United Airlines,914.0
4,7:40p,6:13a,27h 57m,"(ORD, AUH)",Etihad Airways,1242.0
5,8:40a,8:01p,27h 9m,"(LHR, DEL)",Virgin Atlantic,1421.0
6,10:20a,8:01p,28h 49m,"(LHR, DEL)",Virgin Atlantic,1421.0
7,3:45p,8:01p,34h 14m,"(LHR, DEL)",Virgin Atlantic,1421.0
8,5:20p,8:01p,35h 49m,"(LHR, DEL)",Virgin Atlantic,1421.0
9,6:50p,8:01p,37h 19m,"(LHR, BOM)",Virgin Atlantic,1421.0


In [323]:
data_final= data.append(data_su,ignore_index=True)



C:\Users\yagna\AppData\Local\Temp\ipykernel_2240\3110608150.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_final= data.append(data_su,ignore_index=True)


In [324]:
data_final.sort_values('price',ascending= True)

,arrival,depature,duration,layovers,airlines,price
0,15:38,03:15,26h 7m,4h 11m in New York (JFK) • 2h 25m in Abu Dhabi...,Multiple airlines,787.2
1,12:02,03:15,29h 43m,7h 50m in New York (JFK) • 2h 25m in Abu Dhabi...,Multiple airlines,787.2
2,12:02,03:15,29h 43m,7h 50m in New York (JFK) • 2h 25m in Abu Dhabi...,Etihad Airways,787.2
3,11:00,03:15,30h 45m,8h 53m in New York (JFK) • 2h 25m in Abu Dhabi...,Multiple airlines,787.2
4,08:40,03:15,33h 5m,11h 13m in New York (JFK) • 2h 25m in Abu Dhab...,Etihad Airways,787.2
5,6:55a,12:45p,32h 40m,"(BOS, AMS, BOM)",KLM,789.0
6,6:55a,11:20a,34h 5m,"(JFK, CDG, BOM)",Air France,792.0
7,8:20a,11:20a,35h 30m,"(JFK, CDG, BOM)",Air France,792.0
8,3:05a,5:57p,23h 38m,"(EWR, DXB)",United Airlines,914.0
9,7:40p,6:13a,27h 57m,"(ORD, AUH)",Etihad Airways,1242.0
